In [1]:
import ROOT
import root_numpy
from root_numpy import root2array, rec2array, array2hist
import numpy as np
import json
import logging
import time
#sklearn module
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from xgboost import plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score,roc_curve
start=time.time()
logging.basicConfig(format='%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s',
                    filename="check_logs", filemode="w", level=logging.DEBUG)

Welcome to JupyROOT 6.22/02


In [2]:
logging.info("Print config info")
with open("config/config.json","r") as f_obj:
    config = json.load(f_obj)
#print(config)

In [3]:
def Histograms(config, dataset, weights, j, min, max,sample="signal"):
    h=ROOT.TH1F(sample+"_"+config["var_signal"][j],sample+"_"+config["var_signal"][j],100,min,int(max/10+12))
    for i in range(len(dataset)):
        #if i%10000==0 :
#            logging("processing variable "+str(j)+" : "+str(i)+"/"+str(len(dataset)))
#        if i%10000==0 :
#            print(weights[i])
        h.Fill(dataset[i][j],weights[i])
    h.GetXaxis().SetTitle(config["var_name"][j])
    return h

In [5]:
'''
Plot variables
'''
def plotVars(config, signal_dataset, bkg_dataset, signal_weights, bkg_weights):
    j = 0
    for variable in config["var_signal"]:
        max = 0
        min = 0
        for i in range(len(signal_dataset)):
            max=np.maximum(signal_dataset[i][j],max)
            min=np.minimum(signal_dataset[i][j],min)
        for i in range(len(bkg_dataset)):
            max=np.maximum(bkg_dataset[i][j],max)
            min=np.minimum(bkg_dataset[i][j],min)
        logging.info("range of "+variable+" : ["+str(min)+", "+str(max)+"]")
        logging.info("plotting "+variable)
        hist_signal = Histograms(config, signal_dataset, signal_weights, j, min, max,sample="signal")
        hist_bkg    = Histograms(config, bkg_dataset   ,    bkg_weights, j, min, max,sample="bkg"   )
        canvas = ROOT.TCanvas("canvas","canvas",600,600)
        canvas.SetName(config["var_signal"][j])
        hist_signal.Scale(1/hist_signal.Integral())
        hist_bkg.Scale(1/hist_bkg.Integral())
        hist_signal.SetLineColor(2)
        hist_bkg.SetLineColor(4)
        hist_signal.SetMarkerColor(2)
        hist_bkg.SetMarkerColor(4)
        hist_signal.SetFillColor(2)
        hist_bkg.SetFillColor(4)
        hist_signal.SetFillStyle(3004)
        hist_bkg.SetFillStyle(3005)
        ymax= np.maximum(hist_signal.GetMaximum(), hist_bkg.GetMaximum())
        hist_signal.GetYaxis().SetRangeUser(0,1.5*ymax)
        hist_signal.Draw("HIST")
        hist_bkg.Draw("HIST same")
        legend = ROOT.TLegend(.7,.7,.9,.9)
        legend.AddEntry(hist_signal,"signal","F")
        legend.AddEntry(hist_bkg,"background","F")
        legend.Draw()
        canvas.Draw()
        #WorkDir = config["WorkDir"]
        #print(WorkDir+"plots/"+config["var_signal"][j]+".png")
        #canvas.Print(WorkDir+"plot/"+config["var_signal"][j]+".png")
        #print("plots/"+config["var_signal"][j]+".png")
        #canvas.SaveAs("plot/"+config["var_signal"][j]+".png")
        #%jsroot on 
        #canvas.Write()
        #hist_signal.Write()
        #hist_bkg.Write()
        j = j+1
        #print('proccessed', j)

In [7]:
signal_dataset = root2array(filenames=config["signal_file_list"],treename=config["signal_tree_name"],
                          branches=config["var_signal"],selection=config["signal_selection"],include_weight=False)
signal_weights = root2array(  filenames=config["signal_file_list"],treename=config["signal_tree_name"],
                            branches=config["signal_weight_name"],include_weight=False)
bkg_dataset    = root2array( filenames=config["bkg_file_list"],treename=config["bkg_tree_name"],
                        branches=config["var_bkg"],selection=config["bkg_selection"],include_weight=False)
bkg_weights    = root2array(  filenames=config["bkg_file_list"],treename=config["bkg_tree_name"],
                                    branches=config["bkg_weight_name"],include_weight=False)
# Normalising to the total number of events
signal_weights = [1.0/len(signal_dataset) for i in range(len(signal_dataset))]
bkg_weights    = [1.0/len(bkg_dataset) for i in range(len(bkg_dataset))]

#print(type(signal_dataset))
#rint(signal_dataset)
#print(signal_weights)

signal_dataset = rec2array(signal_dataset)
bkg_dataset = rec2array(bkg_dataset)
#rint(signal_dataset)
logging.info("size of signal : "+str(len(signal_dataset)))
logging.info("size of bkg : "+str(len(bkg_dataset)))

# Prepare the data for the traning; 1 for signal and -1 for bkg
target_signal=[1 for i in range(len(signal_dataset))]
target_bkg=[-1 for i in range(len(bkg_dataset))]
X=[]
X.extend(signal_dataset)
X.extend(bkg_dataset)
y=[]
y.extend(target_signal)
y.extend(target_bkg)
weights=[]
weights.extend(signal_weights)
weights.extend(bkg_weights)

# Spilt samples into traning and testing;1/2

signal_train, signal_test= np.split(signal_dataset, [int(len(signal_dataset)*config["test_size"])])
bkg_train, bkg_test= np.split(bkg_dataset, [int(len(bkg_dataset)*config["test_size"])])
target_signal_train = [1 for i in range(len(signal_train))]
target_signal_test = [1 for i in range(len(signal_test))]
target_bkg_train = [-1 for i in range(len(bkg_train))]
target_bkg_test = [-1 for i in range(len(bkg_test))]
signal_weight_train, signal_weight_test= np.split(signal_weights, [int(len(signal_weights)*config["test_size"])])
bkg_weight_train, bkg_weight_test= np.split(bkg_weights, [int(len(bkg_weights)*config["test_size"])])

X_train =[]
X_train.extend(signal_train)
X_train.extend(bkg_train)
X_test =[]
X_test.extend(signal_test)
X_test.extend(bkg_test)
y_train=[]
y_train.extend(target_signal_train)
y_train.extend(target_bkg_train)
y_test=[]
y_test.extend(target_signal_test)
y_test.extend(target_bkg_test)
weight_train=[]
weight_test=[]
weight_train.extend(signal_weight_train)
weight_train.extend(bkg_weight_train)
weight_test.extend(signal_weight_test)
weight_test.extend(bkg_weight_test)

logging.info("size of signal train sample : "+str(len(signal_train)))
logging.info("size of bkg train sample : "+str(len(bkg_train)))
logging.info("size of training sample : "+str(len(X_train)))
logging.info("size of signal test sample : "+str(len(signal_test)))
logging.info("size of bkg test sample : "+str(len(bkg_test)))
logging.info("size of testing sample : "+str(len(X_test)))
logging.info("size of train weight : "+str(len(weight_train)))
logging.info("size of test weight : "+str(len(weight_test)))

#print(signal_dataset)
#print(signal_weights)
#print(bkg_dataset)
#print(bkg_weights)
#plotVars(config, signal_dataset, bkg_dataset, signal_weights, bkg_weights)

In [8]:
def check_model(config, clf, X_train, y_train, X_test, y_test, weight_train, weight_test, method):
#    f=ROOT.TFile("a.root","recreate")
    logging.info("training "+method)
    clf.fit(X_train, y_train, sample_weight=weight_train)
    y_test_pre=clf.predict(X_test)
    y_train_pre=clf.predict(X_train)
    logging.debug("y_train : ")
    logging.debug(y_train)
    logging.debug("y_train_per:")
    logging.debug(y_train_pre)
    logging.debug("y_test:")
    logging.debug(y_test)
    logging.debug("y_test_pre:")
    logging.debug(y_test_pre)
    logging.info("checking the score")
    score = mean_squared_error(y_test, y_test_pre)
    logging.info("mean squared error : "+str(score))

In [9]:
def classfication(config, model_type, X_train, y_train, X_test, y_test, weight_train, weight_test):
####################################################################
#   1,BDT  Not yet sure about these but let's try them. Checkinging if they are same as the BDT, BDTG in TMVA
#   2,BDTG
#   3,XGBoost
#####################################################################
    if "BDT" in model_type:
        #class sklearn.ensemble.AdaBoostRegressor(base_estimator=None, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None
        logging.info("training BDT")
        clf = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(criterion="mse",max_depth=4),n_estimators=850, learning_rate=0.5, )
        logging.info("checking BDT")
        clf.fit(X_train,y_train)
        check_model(config, clf, X_train, y_train, X_test, y_test, weight_train, weight_test,"BDT")
    if "BDTG" in model_type:
        clf = GradientBoostingRegressor(n_estimators=100)
        check_model(config, clf, X_train, y_train, X_test, y_test, weight_train, weight_test,"BDTG")
    if "XGBoost" in model_type:
        clf = xgb.XGBRegressor()
        clf.fit(X_train, y_train)
        check_model(config, clf, X_train, y_train, X_test, y_test, weight_train, weight_test,"XGBoost")


In [10]:
classfication(config,config["methods"],X_train=X_train,y_train=y_train, X_test=X_test, y_test=y_test, weight_train=weight_train, weight_test=weight_test)
end=time.time()
logging.info("using time : "+str(end-start)+" seconds")